# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

from pathlib import Path

import pandas as pd

from ase import io

# #########################################################
from dft_workflow_methods import get_path_rel_to_proj
from dft_workflow_methods import get_job_paths_info
from dft_workflow_methods import get_job_spec_dft_params, get_job_spec_scheduler_params
from dft_workflow_methods import submit_job

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/dft_workflow/bin


# Script Inputs

In [2]:
root_dir = os.getcwd()

compenv = os.environ["COMPENV"]

if compenv == "wsl":
    root_dir = os.path.join(
        os.environ["PROJ_irox_oer_gdrive"],
        "dft_workflow")    
        # "dft_workflow/run_slabs/run_o_covered")    
        # "dft_workflow/run_slabs/run_o_covered/out_data.old/dft_jobs/slac")    


# slac_sub_queue = "suncat"  # 'suncat', 'suncat2', 'suncat3'

In [3]:
# print("sys.argv:", "\n", sys.argv)

print("")
print("sys.argv:")
tmp = [print(i) for i in sys.argv]
print("")
print("What is this one?", sys.argv[-1])
print("")


if sys.argv[-1] == "run":
    run_jobs = True
    print("running job isdjifjsiduf89usd089ufg089sady890gyasd9p8yf978asdy89fyasd89yf890asd7890f7890asd7f89sd")
else:
    run_jobs = False


sys.argv:
/home/raulf2012/anaconda3/envs/PROJ_irox_oer/lib/python3.6/site-packages/ipykernel_launcher.py
-f
/home/raulf2012/.local/share/jupyter/runtime/kernel-febdab31-0590-473b-a034-b48cb70a9b93.json

What is this one? /home/raulf2012/.local/share/jupyter/runtime/kernel-febdab31-0590-473b-a034-b48cb70a9b93.json



# Parse directories

In [4]:
from dft_workflow_methods import parse_job_dirs

df = parse_job_dirs(root_dir=root_dir)

In [5]:
def method(row_i):
    new_column_values_dict = {
        "compenv": None,
        }

    cand_clusters = []
    clusters_list = ["nersc", "sherlock", "slac", ]
    for i in row_i.path_job_root.split("/"):
        if i in clusters_list:
            cand_clusters.append(i)

    if len(cand_clusters) == 1:
        cluster_i = cand_clusters[0]
        new_column_values_dict["compenv"] = cluster_i
    else:
        print("Couldn't parse cluster from path")
        print(cand_clusters)

    # #####################################################
    for key, value in new_column_values_dict.items():
        row_i[key] = value
    return(row_i)

df_i = df
df_i = df_i.apply(
    method,
    axis=1)
df = df_i

In [6]:
from misc_modules.pandas_methods import reorder_df_columns

col_order = [
    "compenv",
    "is_submitted",
    "att_num",
    "rev_num",
    "is_rev_dir",
    "is_attempt_dir",

    "path_full",
    "path_rel_to_proj",
    "path_job_root",
    "path_job_root_w_att_rev",
    "path_job_root_w_att",
    "gdrive_path",
    ]
df = reorder_df_columns(col_order, df)

In [7]:
df_i = df
df_i = df_i[
    (df_i.is_submitted == False) & \
    [True for i in range(len(df_i))]
    ]

df_i

,compenv,is_submitted,att_num,rev_num,is_rev_dir,is_attempt_dir,path_full,path_rel_to_proj,path_job_root,path_job_root_w_att_rev,path_job_root_w_att,gdrive_path,is_empty
280,nersc,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_oh_covered/out_data...,dft_workflow/run_slabs/run_oh_covered/out_data...,dft_workflow/run_slabs/run_oh_covered/out_data...,dft_workflow/run_slabs/run_oh_covered/out_data...,False
282,nersc,False,2,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_oh_covered/out_data...,dft_workflow/run_slabs/run_oh_covered/out_data...,dft_workflow/run_slabs/run_oh_covered/out_data...,dft_workflow/run_slabs/run_oh_covered/out_data...,False
951,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
953,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
962,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
972,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
976,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
1014,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
1022,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False
1031,sherlock,False,1,2,True,True,/media/raulf2012/research_backup/PROJ_irox_oer...,,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,dft_workflow/run_slabs/run_o_covered/out_data/...,False


In [8]:
for i in df_i.path_full.tolist():
    print("rm -r " + i)

rm -r /media/raulf2012/research_backup/PROJ_irox_oer_gdrive/dft_workflow/run_slabs/run_oh_covered/out_data/dft_jobs/nersc/b5cgvsb16w/111/oh/active_site__71/01_attempt/_02
rm -r /media/raulf2012/research_backup/PROJ_irox_oer_gdrive/dft_workflow/run_slabs/run_oh_covered/out_data/dft_jobs/nersc/b5cgvsb16w/111/oh/active_site__71/02_attempt/_02
rm -r /media/raulf2012/research_backup/PROJ_irox_oer_gdrive/dft_workflow/run_slabs/run_o_covered/out_data/dft_jobs/sherlock/7ic1vt7pz4/002/01_attempt/_02
rm -r /media/raulf2012/research_backup/PROJ_irox_oer_gdrive/dft_workflow/run_slabs/run_o_covered/out_data/dft_jobs/sherlock/7ic1vt7pz4/110/01_attempt/_02
rm -r /media/raulf2012/research_backup/PROJ_irox_oer_gdrive/dft_workflow/run_slabs/run_o_covered/out_data/dft_jobs/sherlock/8l919k6s7p/1-100/01_attempt/_02
rm -r /media/raulf2012/research_backup/PROJ_irox_oer_gdrive/dft_workflow/run_slabs/run_o_covered/out_data/dft_jobs/sherlock/926dnunrxf/10-1/01_attempt/_02
rm -r /media/raulf2012/research_backup/

In [9]:
# df_i.iloc[0].to_dict()

In [10]:
# "rclone copy $rclone_gdrive_stanford:norskov_research_storage/00_projects/PROJ_irox_oer/dft_workflow/run_slabs/run_o_covered/out_data/dft_jobs/nersc/b19q9p6k72/101/01_attempt/_02 $PROJ_irox_oer/dft_workflow/run_slabs/run_o_covered/out_data/dft_jobs/b19q9p6k72/101/01_attempt/_02"

# # $rclone_gdrive_stanford
# # raul_gdrive_stanford

In [11]:
# df_i = df[df.is_submitted == False]

# grouped = df_i.groupby(["compenv", ])
# for name, group in grouped:
#     tmp = 42

#     print(40 * "=")
#     print(name)
#     print(40 * "=")
#     for name_i, row_i in group.iterrows():
#         # row_i = group.iloc[0]

#         # #########################################################
#         path_job_root_w_att_rev = row_i.path_job_root_w_att_rev
#         # #########################################################

#         # #########################################################
#         # Constructing path on cluster (remove cluster from path)
#         clust_path_list = []
#         for i in path_job_root_w_att_rev.split("/"):
#             clusters_list = ["nersc", "sherlock", "slac", ]

#             if i not in clusters_list:
#                 clust_path_list.append(i)

#         clust_path = "/".join(clust_path_list)

#         # #########################################################
#         # Constructing Rclone command
#         rclone_comm = "" + \
#             "rclone copy " + \
#             "$rclone_gdrive_stanford:norskov_research_storage/00_projects/PROJ_irox_oer/" + \
#             path_job_root_w_att_rev + \
#             " " + \
#             "$PROJ_irox_oer/" + \
#             clust_path + \
#             ""

#         print(rclone_comm)
#     print("")

In [12]:
# row_i = df.iloc[0]

# cand_clusters = []
# clusters_list = ["nersc", "sherlock", "slac", ]
# for i in row_i.path_job_root.split("/"):
#     if i in clusters_list:
#         cand_clusters.append(i)

# if len(cand_clusters) == 1:
#     cluster_i = cand_clusters[0]
# else:
#     print("Couldn't parse cluster from path")
#     print(cand_clusters)

# cluster_i